## FID with Numpy

In [1]:
import numpy as np
from scipy.linalg import sqrtm

def calculate_fid(act1, act2):
  # Calculate mean and covariance statisctics
  mu1, cov1 = act1.mean(axis=0), np.cov(act1, rowvar=False)
  mu2, cov2 = act2.mean(axis=0), np.cov(act2, rowvar=False)
  # Calculate sum squared difference between means
  ssdiff = np.sum((mu1 - mu2)**2.0)
  # Calculate sqrt of product between cov
  covmean = sqrtm(cov1.dot(cov2))
  if np.iscomplexobj(covmean):
    covmean = covmean.real
  fid = ssdiff + np.trace(cov1 + cov2 - 2.0 * covmean)
  return fid

act1 = np.random.random(10*2048)
act1 = act1.reshape((10, 2048))
act2 = np.random.random(10*2048)
act2 = act2.reshape((10, 2048))

fid = calculate_fid(act1, act1)
print('FID (same): %.3f' % fid) 
# fid between act1 and act2 
fid = calculate_fid(act1, act2)
print('FID (different): %.3f' % fid)

FID (same): -0.000
FID (different): 357.684


## FID with Keras

In [2]:
from tensorflow import keras
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from skimage.transform import resize

def scale_images(images, new_shape):
  images_list = list()
  for image in images:
    new_image = resize(image, new_shape, 0)
    images_list.append(new_image)
  return np.asarray(images_list)

def calculate_fid(model, images1, images2):
  # Calculate activations
  act1 = model.predict(images1)
  act2 = model.predict(images2)
  # Calculate mean and covariance statisctics
  mu1, cov1 = act1.mean(axis=0), np.cov(act1, rowvar=False)
  mu2, cov2 = act2.mean(axis=0), np.cov(act2, rowvar=False)
  # Calculate sum squared difference between means
  ssdiff = np.sum((mu1 - mu2)**2.0)
  # Calculate sqrt of product between cov
  covmean = sqrtm(cov1.dot(cov2))
  if np.iscomplexobj(covmean):
    covmean = covmean.real
  fid = ssdiff + np.trace(cov1 + cov2 - 2.0 * covmean)
  return fid

# model = InceptionV3(include_top=False, pooling='avg', input_shape=(299, 299, 3))
# images1 = np.random.randint(0, 255, 10*32*32*3)
# images1 = images1.reshape((10, 32, 32, 3))
# images2 = np.random.randint(0, 255, 10*32*32*3)
# images2 = images2.reshape((10, 32, 32, 3))
# print('Prepared', images1.shape, images2.shape)
# images1 = images1.astype('float32')
# images1 = images1.astype('float32')

# images1 = scale_images(images1, (299, 299, 3))
# images2 = scale_images(images2, (299, 299, 3))
# print('Scaled', images1.shape, images2.shape)

# images1 = preprocess_input(images1)
# images2 = preprocess_input(images2)

# fid = calculate_fid(model, images1, images1)
# print('FID (same): %.3f' % fid)

# fid = calculate_fid(model, images1, images2) 
# print('FID (different): %.3f' % fid)

In [3]:
model = InceptionV3(include_top=False, pooling='avg', input_shape=(299, 299, 3))
(images1, _), (images2, _) = keras.datasets.cifar10.load_data()
np.random.shuffle(images1)
images1 = images1[:2000]
images2 = images2[:2000]
print('Loaded', images1.shape, images2.shape)
images1 = images1.astype('float32')
images2 = images2.astype('float32')
images1 = scale_images(images1, (299, 299, 3))
images2 = scale_images(images2, (299, 299, 3))
print('Scaled', images1.shape, images2.shape)
images1 = preprocess_input(images1)
images2 = preprocess_input(images2)
fid = calculate_fid(model, images1, images2) 
print('FID: %.3f' % fid)

170498071/170498071 [==============================] - 2s 0us/step
Loaded (2000, 32, 32, 3) (2000, 32, 32, 3)
Scaled (2000, 299, 299, 3) (2000, 299, 299, 3)
63/63 [==============================] - 7s 112ms/step
FID: 26.353
